# Task description

## NoSQL data - Spark and PySpark

1. Choose a data source that comes in any text format, and this time, it must not be as strict as the data format for a relational database. E.g. your data might have varying numbers of keys, varying types of values, etc. If you don’t have an own data source, try to save data from an API that gives you large JSON responses such as the Twitter Streaming API.
2. Download a pre-installed version of Spark on your own computer or on a remote computer and install and link PySpark to the Spark executables: https://medium.com/tinghaochen/how-to-install-pyspark-locally-94501eefe421
Demonstrate that you can access your Spark from a Jupyter Notebook by importing pyspark and creating your first SparkContext object. If possible, have a look at the logs in the terminal of the notebook server. Make a screenshot about the jvm engine.
3. Go to the localhost port where you can graphically access the Spark jobs. Try to understand what you have there, make a screenshot.
Read in your textfiles into a SparkContext object, create your first Spark pipeline where you
- - Convert the text row into meaningful variables (e.g. splitting a csv line at commas, then converting the classes of the strings you got to int/float what you need).
-- Filter your rows
-- Calculate at least one aggregated result with the reduceByKey method.
-- Check each step by using the first(n) or take(n) operations that limit the number of rows you fetch from the SparkContext instance!
4. Advanced: experiment with flatMap!
5. Either collect your results into memory with the .collect() method, or write them out as textfiles. 
6. Create a visualization that demonstrates a task that you solved with PySpark.
7. Check parallel processor usage with htop while running your Spark job. Make a screenshot about Spark running on all of your cores!

Document your workflow in a git repository. You should turn in the link of the git repo in the end. Create a README.md in the root of the repo that contains the source and the documentation of the data, and contains your notes about the whole process. Include the screenshots and what you learned from the task.

---

My main computer is a mac, so I set up a working system on Mac OS, which is sort of similiar to a linux distribution, with some differences, that I will pinpoint.

Since on the current system (Big Sur - 2020 December) Python is somewhat bugged, I had to install a separate environment, I chose Miniconda. I tried to use this guide, I had to adjust it in minor ways: https://medium.com/macoclock/how-to-install-apache-pyspark-on-macbook-pro-4a9249f0d823

# Installing main dependencies

1. Miniconda was as easy as downloading an installer, and installing it. https://docs.conda.io/projects/conda/en/latest/user-guide/install/macos.html

2. Java was again, easy, just had to download and install the latest Java Runtime: https://www.java.com/en/download/ (Version 8 Update 271)

3. Installing PySpark. This is where I used conda for the first time:
conda install pyspark

4. Installing FindSpark:
conda install -c conda-forge findspark

5. Installing Jupyter Notebook:
conda install notebook

# Checking things if they work

1. Started Jupyter Notebook typing "jupyter notebook" in the Terminal.

2. Trying if it works:


In [1]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
import random

import sys

In [20]:
sc = SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

3.14163748


![title](test_terminal.png)

After that I could check the following: http://localhost:4040/ where I could see that it correctly performed the operations.

![title](test_jobs.png)

This was a sample task that I found on the internet, it calculates the value of pi using a monte-carlo simulation method. Looking at Spark Jobs page we can see a counting job that was performed, hence "Completed", the running time was 13 seconds, and all the stages (8 stages - 8 lines) was completed successfully.

In [17]:
sc.stop()

---

# Creating a Spark Context for receiving Twitter data

At this point I had to make 2 separate notebooks, one for the Spark setup and handling data, the other for the Twitter Stream API.

I created the pipeline for the datastream, so I can receive tweets, and pipe them into the created Spark Context.

In [2]:
# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")
# read data from port 37346
dataStream = ssc.socketTextStream("localhost", 37346)

After setting up the basic datastream comes handling the data and starting the stream. Here I split the tweets into words, filtered the words for hashtags, and calculated each hashtag's occurence.

In [3]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

In [4]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']
def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [ ]:
# split each tweet into words
words = dataStream.flatMap(lambda line: line.decode('utf-8').split(" "))
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

----------- 2021-01-14 19:56:00 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:02 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:04 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:06 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:08 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:10 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:12 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:14 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:16 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:18 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:20 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:22 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:24 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:56:26 -----------
Error: <class 'NameError'>
------

----------- 2021-01-14 19:59:52 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:59:54 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:59:56 -----------
Error: <class 'NameError'>
----------- 2021-01-14 19:59:58 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:00 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:02 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:04 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:06 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:08 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:10 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:12 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:14 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:16 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:00:18 -----------
Error: <class 'NameError'>
------

----------- 2021-01-14 20:03:44 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:03:46 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:03:48 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:03:50 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:03:52 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:03:54 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:03:56 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:03:58 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:04:00 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:04:02 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:04:04 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:04:06 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:04:08 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:04:10 -----------
Error: <class 'NameError'>
------

----------- 2021-01-14 20:07:36 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:38 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:40 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:42 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:44 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:46 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:48 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:50 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:52 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:54 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:56 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:07:58 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:08:00 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:08:02 -----------
Error: <class 'NameError'>
------

----------- 2021-01-14 20:11:28 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:30 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:32 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:34 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:36 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:38 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:40 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:42 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:44 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:46 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:48 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:50 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:52 -----------
Error: <class 'NameError'>
----------- 2021-01-14 20:11:54 -----------
Error: <class 'NameError'>
------

...

----------- 2021-01-15 08:37:18 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:20 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:22 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:24 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:26 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:28 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:30 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:32 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:34 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:36 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:38 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:40 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:42 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:37:44 -----------
Error: <class 'NameError'>
------

----------- 2021-01-15 08:41:10 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:12 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:14 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:16 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:18 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:20 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:22 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:24 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:26 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:28 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:30 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:32 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:34 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:41:36 -----------
Error: <class 'NameError'>
------

----------- 2021-01-15 08:45:10 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:12 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:14 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:16 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:18 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:20 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:22 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:24 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:26 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:28 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:30 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:32 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:34 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:45:36 -----------
Error: <class 'NameError'>
------

----------- 2021-01-15 08:49:04 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:06 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:08 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:10 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:12 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:14 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:16 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:18 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:20 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:22 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:24 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:26 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:28 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:49:30 -----------
Error: <class 'NameError'>
------

----------- 2021-01-15 08:52:56 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:52:58 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:00 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:02 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:04 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:06 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:08 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:10 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:12 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:14 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:16 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:18 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:20 -----------
Error: <class 'NameError'>
----------- 2021-01-15 08:53:22 -----------
Error: <class 'NameError'>
------

# References
1. https://medium.com/macoclock/how-to-install-apache-pyspark-on-macbook-pro-4a9249f0d823
2. https://requests-oauthlib.readthedocs.io/en/latest/oauth1_workflow.html
3. https://www.toptal.com/apache/apache-spark-streaming-twitter